In [2]:
import pandas as pd

**Data has no heading, so we need to give names to the columns**

In [3]:
columns = ['stock_identifier', 'col_2', 'bid_price', 'ask_price', 
           'trade_price', 'bid_volume', 'ask_volume', 'trade_volume', 
           'update_type', 'col_10', 'date','time_seconds', 'open_price', 
           'col_14', 'condition_codes_1', 'condition_codes_2']

**As this is an extremely large file, it will be imported in chunks**

In [4]:
scanditick = pd.read_csv("/Users/xiac/Documents/Yandex/scandi.csv", 
                         names = columns, chunksize = 50000)

**Concat the chunks to make one complete dataframe**

In [5]:
scandi = pd.concat(scanditick, ignore_index = True)

In [110]:
scandi.head(5)

,stock_identifier,col_2,bid_price,ask_price,trade_price,bid_volume,ask_volume,trade_volume,update_type,col_10,date,time_seconds,open_price,col_14,condition_codes_1,condition_codes_2
0,BBHBEAT Index,59,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28264.0,0.0,0,@1,NaN
1,BBHBEAT Index,60,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28265.0,0.0,0,@1,NaN
2,BBHBEAT Index,61,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28266.0,0.0,0,@1,NaN
3,BBHBEAT Index,62,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28267.0,0.0,0,@1,NaN
4,BBHBEAT Index,63,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28268.0,0.0,0,@1,NaN


In [7]:
scandi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13260277 entries, 0 to 13260276
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   stock_identifier   object 
 1   col_2              int64  
 2   bid_price          float64
 3   ask_price          float64
 4   trade_price        float64
 5   bid_volume         int64  
 6   ask_volume         int64  
 7   trade_volume       int64  
 8   update_type        int64  
 9   col_10             int64  
 10  date               int64  
 11  time_seconds       float64
 12  open_price         float64
 13  col_14             int64  
 14  condition_codes_1  object 
 15  condition_codes_2  object 
dtypes: float64(5), int64(8), object(3)
memory usage: 1.6+ GB


**Filter the data as required**

In [105]:
scandi_sub = scandi[(scandi['condition_codes_1'].isna()) | 
                    (scandi['condition_codes_1'] == "XT")]

In [106]:
len(scandi_sub)

13122036

In [13]:
scandi_sub = scandi_sub[scandi_sub['bid_price'] <= scandi_sub['ask_price']]

In [16]:
scandi_sub = scandi_sub.drop(['col_2', 'col_10', 'col_14'], axis = 1)

**define the functions that will go through each stock and get the last_digit_ratio, as well as time_between_trades stats**

In [140]:
def last_digit_ratio(stock_identifier, column):
    #change column to a list
    num_list = scandi_sub[scandi_sub['stock_identifier'] == stock_identifier][column].tolist()
    #change values in list to strings
    string_list = [repr(n) for n in num_list]
    #take the last item in each string then convert to integer
    last_digit_list = [int(n[-1]) for n in string_list]
    #calcuate the ratio
    ratios_dict = {}
    for i in range(10):
        ratio = last_digit_list.count(i) / len(last_digit_list) 
        ratios_dict[i] = ratio
    return ratios_dict

In [131]:
#dataset is named 'scandi', and columns named 'stock_identifer' and 'update_type'
def time_difference_stats(stock_identifier):
    from statistics import mean, median
    
    data_sub = scandi_sub[(scandi_sub['stock_identifier'] == stock_identifier) & 
                      (scandi_sub['update_type'] == 1)]
    time_seconds_list = data_sub['time_seconds'].tolist()
    
    time_difference_list = [time_seconds_list[i+1] - 
                        time_seconds_list[i] for i in range(len(time_seconds_list)-1) 
                            if time_seconds_list[i+1] > time_seconds_list[i]]

    trade_time_mean = mean(time_difference_list)
    trade_time_median = median(time_difference_list)
    trade_time_max = max(time_difference_list)
    d = {'trade_time_mean': trade_time_mean, 'trade_time_median': trade_time_median,
            'trade_time_max': trade_time_max}
    return d

**Create a list that has all the stock identifiers**

In [132]:
stock_list = scandi_sub['stock_identifier'].unique()

**create the dataframe that contains time between trades stats for all stocks**

In [133]:
time_difference_results = []
for x in stock_list:
    time_difference_results.append(time_difference_stats(x))


In [134]:
time_between_trades = pd.DataFrame(index = stock_list, 
                                           data = time_difference_results,
                                          columns = ['trade_time_mean', 'trade_time_median',
                                                    'trade_time_max'])

In [135]:
time_between_trades

,trade_time_mean,trade_time_median,trade_time_max
ABB SS Equity,33.148016,17.0,490.0
ALFA SS Equity,20.590654,5.0,447.0
ASSAB SS Equity,38.005960,15.0,537.0
ATCOA SS Equity,21.254386,10.0,349.0
ATCOB SS Equity,47.186989,24.0,611.0
...,...,...,...
STL NO Equity,17.349556,7.0,310.0
SUBC NO Equity,25.567135,11.0,639.0
TEL NO Equity,35.223965,17.0,549.0
TGS NO Equity,38.514244,13.0,879.0


**create the dataframe that contains the last digits ratios for trade prices of all stocks**

In [141]:
last_digit_results = []
for x in stock_list:
    last_digit_results.append(last_digit_ratio(x, 'trade_price'))

In [142]:
last_digit_ratio_allstocks = pd.DataFrame(index = stock_list, 
                                          data = last_digit_results,
                                         columns = [0,1,2,3,4,5,6,7,8,9])

In [143]:
last_digit_ratio_allstocks

,0,1,2,3,4,5,6,7,8,9
ABB SS Equity,0.122550,0.082505,0.075844,0.066620,0.072543,0.087164,0.094688,0.119814,0.136715,0.141557
ALFA SS Equity,0.133117,0.100069,0.108822,0.078165,0.085533,0.099690,0.089896,0.100028,0.107754,0.096926
ASSAB SS Equity,0.489653,0.000000,0.000000,0.000000,0.000000,0.510347,0.000000,0.000000,0.000000,0.000000
ATCOA SS Equity,0.113704,0.095195,0.094416,0.097632,0.093628,0.104028,0.099605,0.098339,0.098600,0.104853
ATCOB SS Equity,0.112756,0.098566,0.088869,0.106744,0.114364,0.111246,0.109098,0.089579,0.080763,0.088015
...,...,...,...,...,...,...,...,...,...,...
STL NO Equity,0.121869,0.106839,0.108069,0.090986,0.104714,0.096019,0.082126,0.080030,0.098330,0.111018
SUBC NO Equity,0.055493,0.061134,0.060146,0.060360,0.054745,0.524606,0.048339,0.048998,0.038152,0.048028
TEL NO Equity,0.115531,0.109147,0.117178,0.114402,0.091162,0.078582,0.082731,0.089727,0.102606,0.098935
TGS NO Equity,0.189139,0.124039,0.124742,0.099231,0.078774,0.078869,0.067235,0.087422,0.077045,0.073505
